# A program to store vocabulary while learning or practicing a new language.

First, I define a help function `countWords` to determine whether an entry has one, two, or more words.
To test it, I use the array `my_words`.

In [200]:
my_words = ['abarbeiten', 'die Wertschöpfung', 'etwas in Worte fassen']

def count_words(text):
    if len(text) == 0:
        return 0
    words = 1
    for letter in text:
        if letter == ' ' and text[0] != letter and text[-1] != letter:
            words += 1
    return words

for i in my_words:
    print(count_words(i))

# Function to show double-digits for days and months
def adjust_digits(date):
    if len(f'{date}') == 1:
        return f'0{date}'
    else:
        return f'{date}'

def print_abort(message):
    print(message)
    return None

1
2
4


I use the `json` library to create my vocabulary list.

In [202]:
import json

## Description of my classes

1. `Entry`. It has three attributes:
    1. `Text`.
    5. `Date`.
    4. `Category`.
3. `Noun`. A subclass of `Entry`, it has one additional attribute:
    * `Article`.
50. `Dictionary`. It has the following properties:
    + It consists of entries. Those are stored in a JSON file whose path is given to the constructor.
    + Should be able to add entries.
    + It should allow the filtering of the whole list and create sublists using different criteria.

Remark: an `entry` might be a word or a sentence (e.g. a common idiom).

In [205]:
class Entry:
    def __init__(self, text, date, category):
        self.text = text
        self.date = date
        self.category = category
    def __str__(self):
        return self.text
    def __repr__(self):
        return f"{self.text}"

class Noun(Entry):
    def __init__(self, article, text, date, category):
        super().__init__(text, date, category)
        self.article = article
    def __repr__(self):
        return f"{self.article} {self.text}"



In [206]:
# Codigo Clase Diccionario
class Dictionary:
    def __init__(self, file_path):
        self.file_path = file_path
        self.list = self.create_list()

    def create_list(self):
        with open(self.file_path, 'r') as file:
            my_entries = json.loads(file.read())
            #print(mis_entradas)
            entries_categorized = []
            for entry in my_entries["Entradas"]:
                #print(entrada["palabra"])
                if count_words(entry["palabra"]) == 2:
                    entries_categorized.append(Noun(article=entry["palabra"][0:3],
                    text=entry["palabra"][4:], date=entry["fecha"], category="Noun"))
                else:
                    entries_categorized.append(Entry(text=entry["palabra"],
                    date=entry["fecha"], category="Other"))
            return entries_categorized


In [207]:
class VocabularyList:
    def __init__(self, vocab_list):
        self.vocab_list = vocab_list
        self.table = self.create_table()
    
# Create the tabla using the entries in the the dictionary
    def create_table(self):
        numbering = []
        vocabulary = []
        i = 1
        entry_i = "Entry "
        for entry in self.vocab_list:
            numbering.append(entry_i + str(i))
            # TODO: add something to deal with wrong entries
            day = adjust_digits(entry.date[0])
            month = adjust_digits(entry.date[1])
            show_date = day+'-'+month+f'-{entry.date[2]}'
            list_row = [entry.__repr__(), show_date, entry.category]
            vocabulary.append(list_row)
            i += 1
        return vocabulary, numbering, ["", "Text", "Date", "Category"]

# Function to show a well-ordered table with my vocabulary   
    def show_table(self, vocabulary, enumeration, columns):
        # TODO: here, I have to complete the function specifying the desired format
        print("{:<10} {:<40} {:<12} {:<12}".format(*columns))
        # TODO: here handle with the case that enumeration and vocabulary have different lengths
        for i in range(len(vocabulary)):
            print("{:<10} {:<40} {:<12} {:<12}".format(enumeration[i], *vocabulary[i]))

In [231]:
#def isSustantivo(entrada):
 #  if contarPalabras(entrada.texto) == 2:
  #    return true

# function that creates and prints the resulting table
def list_entries(vocab_list):
    vocab_table = VocabularyList(vocab_list)
    table_elements = vocab_table.create_table()
    vocab_table.show_table(table_elements[0], table_elements[1], table_elements[2])

# this function redirects the filtering to the respective method
def filter_entries(vocab_list, attribute, value):
    match attribute:
        case "category": return category_filter(vocab_list, value)
        case "initial_letter": return letter_filter(vocab_list, value)
        case "year" | "month" | "day":
            if value < 0:
                return print_abort("Please enter a positive number!")
            return date_filter(vocab_list, attribute, value)
        case "monthly" | "date":
            try:
                for i in value:
                    if i < 0:
                        print("Please enter positive numbers!")
                        return None
                return date_filter(vocab_list, attribute, value)
            except TypeError as e:
                print(e)
        case _:
            return print_abort("No method for that attribute")

def category_filter(vocab_list, value):
    value = value.lower()
    match value:
        case "noun" | "other": 
            filtered_vocabulary = []
            for i in vocab_list:
                if i.category.lower() == value:
                    filtered_vocabulary.append(i)
            list_entries(filtered_vocabulary)
        case _:
            return print_abort("Invalid category!")

def letter_filter(vocab_list, value):
    filtered_vocabulary = []
    if len(value) != 1:
        return print_abort("Please type only one letter!")
    value = value.lower()
    for i in vocab_list:
        if i.category == "Noun" and i.text[0].lower() == value:
            filtered_vocabulary.append(i)
        elif i.text[0] == value:
            filtered_vocabulary.append(i)
    list_entries(filtered_vocabulary)

def date_filter(vocab_list, attribute, value):
    filtered_vocabulary = []
    #print(value)
    match attribute:
        case "year":
            for i in vocab_list:
                if i.date[2] == value:
                    filtered_vocabulary.append(i)
        case "month":
            for i in vocab_list:
                if value == 0 or value > 12:
                    return print_abort("Invalid month number!")
                elif i.date[1] == value:
                    filtered_vocabulary.append(i)
        case "day":
            for i in vocab_list:
                if value == 0 or value > 31:
                    return print_abort("Invalid day number!")
                if i.date[0] == value:
                    filtered_vocabulary.append(i)
        case "monthly":
            if len(value) != 2:
                return print_abort("Please enter a day number followed by a month number")
            elif value[1] == 0 or value[1] > 12 or value[0] == 0 or value[0] > 31:
                return print_abort("Invalid numbers!")
            else:
                for i in vocab_list:
                    if i.date[0] == value[0] and i.date[1] == value[1]:
                        filtered_vocabulary.append(i)
        case "date":
            if len(value) != 3:
                return print_abort("Please enter a day number followed by a month number and a year")
            elif value[1] == 0 or value[1] > 12 or value[0] == 0 or value[0] > 31:
                return print_abort("Invalid numbers!")
            else:
                for i in vocab_list:
                    if i.date == value:
                        filtered_vocabulary.append(i)
        case _:
            return print_abort("Invalid category!")
    list_entries(filtered_vocabulary)


In [233]:
a = [3,4,5]
type(a)

list

In [235]:
new_dict = Dictionary("entradas_reducidas.json")
my_entries = new_dict.list
print(my_entries)

[abarbeiten, abfangen, die Abfolge, abgelten, abgrenzen, die Besteuerung, bestmoeglich, bewirten, durchreichen, einheitlich, einordnen, der Einsatz, einsortieren, entbehrlich, das Entgelt, entkoppeln, das Fachkonzept, fachlich, die Fachlichkeit, die Festlegung, das Formelwerk, foermlich, gegenueberstellen, misslingen, nachgelagert, nahelegen, niederdruecken, notduerftig, die Plausibilisierung, raushauen, die Rechnungslegung, die Rentenperiode, das Tafelwerk, ungemein, unterlassen, unuebersichtlich, unzulaenglich, verbindlich, verkomplizieren, verschaffen, vervollstaendigen, die Verweildauer, der Vorgang, das Vorhaben, vorlaeufig, der Zeitwert, zusammensetzen, der Zusatz, die Zustaendigkeit, die Zusicherung, zuweisen, zwangslaeufig]


In [237]:
list_entries(my_entries)

           Text                                     Date         Category    
Entry 1    abarbeiten                               29-11-2022   Other       
Entry 2    abfangen                                 24-05-2023   Other       
Entry 3    die Abfolge                              12-12-2022   Noun        
Entry 4    abgelten                                 28-12-2022   Other       
Entry 5    abgrenzen                                24-05-2023   Other       
Entry 6    die Besteuerung                          07-12-2022   Noun        
Entry 7    bestmoeglich                             13-12-2022   Other       
Entry 8    bewirten                                 29-12-2022   Other       
Entry 9    durchreichen                             13-12-2022   Other       
Entry 10   einheitlich                              21-12-2022   Other       
Entry 11   einordnen                                14-12-2022   Other       
Entry 12   der Einsatz                              15-12-2022  

In [239]:
#filter_entries(my_entries, "category", "Other")
#filter_entries(my_entries, "initial_letter", "z")
#filter_entries(my_entries, "year", 2023)
filter_entries(my_entries, "corneta", 20)
filter_entries(my_entries, "year", -2023)
filter_entries(my_entries, "monthly", [4,1])
filter_entries(my_entries, "date", [7,12,2022])


No method for that attribute
Please enter a positive number!
           Text                                     Date         Category    
Entry 1    einsortieren                             04-01-2023   Other       
Entry 2    verkomplizieren                          04-01-2023   Other       
           Text                                     Date         Category    
Entry 1    die Besteuerung                          07-12-2022   Noun        
Entry 2    das Entgelt                              07-12-2022   Noun        
Entry 3    unuebersichtlich                         07-12-2022   Other       


## Next steps:

- Filter using different criteria:
  1. ~~Alphabetic~~
  2. ~~Date~~
  3. ~~Category~~
- Add vocabulary into the JSON file.

In [214]:
def reordenar(lista):
  lista[-1] = -5
  return lista

In [215]:
lista = [1,2,3,4]
reordenar(lista)
print(lista)

[1, 2, 3, -5]


In [216]:
p = "ab"
len(p)

2

In [1]:
lista1 = ["perro", "gato", "raton"]
"pollo" in lista1

False